In [2]:
import os
import sys
import copy
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
sys.path.append("./../../src")

from load_data import gen_data

In [2]:
data = pd.read_csv("./../../data/fraud_detection_bank_dataset.csv")
col_names = [f"col_{i}" for i in range(111)]
col_names_naive = [x for x in col_names if len(data[x].value_counts().keys()) < 10]
target = "targets"

In [3]:
train_data, test_data = train_test_split(data, train_size=0.8, random_state=123)
X_train, y_train = train_data[col_names_naive].values, train_data[target].values
X_test, y_test = test_data[col_names_naive].values, test_data[target].values

In [4]:
def accuracy(y, y_hat):
    
    return (y == y_hat).sum() / len(y)

## KNN Origin

In [5]:
class KNN():
    
    @staticmethod
    def distance(p1, p2):
        
        if p1 is None or p2 is None:
            return 0
        return ((p2 - p1) ** 2).sum() ** 0.5
    
    def __init__(self, top_k = 3):
        
        self.top_k = top_k
        
    def fit(self, X_train, y_train):
        
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        
        dist = []
        for X, y in zip(X_train, y_train):
            
            dist.append((self.distance(X_test, X), y))
            
        rst = sorted(dist, key=lambda x:x[0])[0:self.top_k]
        
        # return rst
        return stats.mode([x[1] for x in rst])[0][0]
        
        

In [6]:
knn = KNN(3)

knn.fit(X_train, y_train)

In [7]:
y_test_hat = np.apply_along_axis(knn.predict, axis=1, arr=X_test)

In [8]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

print("Test confusion matrix: \n", confusion_matrix(y_test, y_test_hat))
print("\nTest precision: ", precision_score(y_test, y_test_hat))
print("Test recall: ", recall_score(y_test, y_test_hat))
print("Test F1: ", f1_score(y_test, y_test_hat))
print("Test accuracy: ", accuracy_score(y_test, y_test_hat))

Test confusion matrix: 
 [[2751  296]
 [ 354  693]]

Test precision:  0.7007077856420627
Test recall:  0.66189111747851
Test F1:  0.6807465618860511
Test accuracy:  0.8412310698583293


## KNN (Kd Tree)

### kd tree

In [192]:
X_train = gen_data(0, 5, 100000, 5, seed=123)

np.shape(X_train)

(100000, 5)

In [233]:
class Node():
    
    def __init__(self):
        self.father = None
        self.left = None
        self.right = None
        self.feature = None
        self.split = None
        
    def _str_(self):
        feature, split = self.feature, self.split
        print(f"Feature:{feature}, Split_value:{split}")
        
    @property
    def brother(self):
        if self.father is None:
            ret = None
        else:
            if self.father.left is self:
                ret = self.father.right
            else:
                ret = self.father.left
        return ret
    
    
class KDTree():
    
    def __init__(self, k=3):
        self.root = Node
        self.k = k
        
    @staticmethod
    def distance(p1, p2):
        if p1 is None or p2 is None:
            return 0
        return ((p2 - p1) ** 2).sum() ** 0.5
    
    @staticmethod
    def _split_feat(X):
        
        feat_idx = np.argmax(X.var(axis=0))
        # print(len(X), feat_idx)

        if len(X) % 2 == 0:
            mid = int(len(X) / 2) - 1
        else:
            mid = int(len(X) / 2 - 0.5)
        
        X_sort = X[X[:,feat_idx].argsort()]
        split = X_sort[:][mid]
        X_left = X_sort[:mid]
        X_right = X_sort[mid+1:]
        
        return feat_idx, split, X_left, X_right
    
    def _search(self, X_i, node):
        
        """
        Search Xi from the KDTree until Xi is at an leafnode.
        
        Arguments:
            Xi[list] -- 1d list with int or float.
        
        Returns:
            node -- Leafnode.
        """
        
        while node.left or node.right:

            # print("feat_idx:{}, Xi value:{}, Node value:{}".format(node.feature, X_i[node.feature], node.split))
                    
            if node.left is None:
                node = node.right
                direct = 'right'
            elif node.right is None:
                node = node.left
                direct = 'left'
            else:
                if X_i[node.feature] < node.split[node.feature]:
                    node = node.left
                    direct = 'left'
                else:
                    node = node.right
                    direct = 'right'
                    
            # print(f"searching path turn {direct}")
        
        # print("feat_idx:{}, Xi value:{}, Node value:{}".format(node.feature, X_i[node.feature], node.split))
        return node
    
    def _get_dist_eu(self, X_i, node):
        
        """
        Calculate euclidean distance between Xi and node.
        
        Arguments:
            Xi[list] -- 1d list with int or float.
            nd[node]
            
        Returns:
            float -- Euclidean distance.
        """

        
        return self.distance(X_i, node.split)
    
    def _get_dist_hyper(self, X_i, node):
        
        """
        Calculate euclidean distance between Xi and hyper plane.
        
        Arguments:
            Xi[list] -- 1d list with int or float.
            nd[node]
            
        Returns:
            float -- Euclidean distance.
        """
        
        feat_idx = node.feature
        
        return abs(X_i[feat_idx] - node.split[feat_idx])
    
    def _backtrace(self, X_i, node, root):
        
        """
        Backtrace node from the KDTree begining with the leafnode.
        
        Arguments:
            Xi]list] -- 1d list with int or float.
        
        Returns:
            node -- The nearest node to Xi.
        """
        
        dist_max = self.kbest[-1][1]
        # print(X_i, node.split, root.split)
        
        while node is not root:
                        
            dist = self._get_dist_eu(X_i, node)
            # print(self.kbest, dist, dist_max)

            if dist < dist_max:
                self.kbest[-1][1] = dist
                self.kbest[-1][0] = node
                self.kbest = sorted(self.kbest, key=lambda x:x[1])
                dist_max = self.kbest[-1][1]
                        
            node_father = node.father
            
            if self._get_dist_hyper(X_i, node_father) <= dist_max:
                return [(node.brother, node.brother, "f"), (node_father, root, "b")]
            else:
                node = node_father
            
        dist = self._get_dist_eu(X_i, node)
        if dist < dist_max:
            self.kbest[-1][1] = dist
            self.kbest[-1][0] = node
            self.kbest = sorted(self.kbest, key=lambda x:x[1])
        return []
    
    def show(self):
        
        queue = [self.root]
        
        while queue:
            nd = queue.pop(0)
            feature, split = nd.feature, nd.split
            print(f"Feature:{feature}, Split_value:{split}")
            if nd.left is not None:
                queue.append(nd.left)
            if nd.right is not None:
                queue.append(nd.right)
                
    def stat_tree(self):
        
        queue = [(self.root, 0)]
        max_depth = -1
        leaf_num = 0
        
        while queue:
            
            node, depth = queue.pop(0)
            
            if depth > max_depth:
                max_depth = depth
                
            if node.left is not None:
                queue.append((node.left, depth+1))    
            
            if node.right is not None:
                queue.append((node.right, depth+1))
                
            if node.left is None and node.right is None:
                leaf_num += 1
            
        print(f"Max_depth of KD-tree: {max_depth}, Leaf number of KD-tree: {leaf_num}")

    
    def build_tree(self, X):
        
        nd = self.root
        queue = [(nd, X)]
        
        while queue:
            
            nd, X = queue.pop(0)
            nd.feature, nd.split, X_left, X_right = self._split_feat(X)
            # print(nd.feature, nd.split, len(X_left), len(X_right))
            
            if len(X_left) != 0:
                nd.left = Node()
                nd.left.father = nd
                queue.append((nd.left, X_left))

            if len(X_right) != 0:
                nd.right = Node()
                nd.right.father = nd
                queue.append((nd.right, X_right))
    
    def search_knearest(self, X_i):
        
        self.kbest = [[None, np.inf] for _ in range(self.k)]
        
        node = self.root
        queue = [(node, node, "f")]
        
        while queue:
    
            current, root, sign = queue.pop(0)
            
            if current is None:
                continue
            
            if sign == "f":
                current = self._search(X_i, current)
                queue += self._backtrace(X_i, current, root)
            elif sign == "b":
                queue += self._backtrace(X_i, current, root)
            
            # print(self.kbest)
            
            

def exhuast_search(X_i, X_train):
    
    def distance(p1, p2):
        if p1 is None or p2 is None:
            return 0
        return ((p2 - p1) ** 2).sum() ** 0.5

    ds = []
    for i in X_train:
        d = distance(i, X_i)
        ds.append([X_i, i, d])
        
    ds = sorted(ds, key=lambda x:x[2])
    
    return ds

In [209]:
kd_tree = KDTree()

kd_tree.build_tree(X_train)

kd_tree.stat_tree()

Max_depth of KD-tree: 16, Leaf number of KD-tree: 34465


In [242]:
import time

print("Result of KD search ...")

X_i = X_train[1100]

time1 = time.time()
kd_tree.search_knearest(X_i)
time2 = time.time()
print("Time consumption:{}".format(time2 - time1))
display([(X_i, x[0].split, x[1]) for x in kd_tree.kbest])

print("\r\nResult of violate search ...")
time1 = time.time()
sds = exhuast_search(X_i, X_train)[:3]
time2 = time.time()
print("Time consumption:{}".format(time2 - time1))
display(sds)


Result of KD search ...
Time consumption:0.00177001953125


[(array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  0.0),
 (array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  array([4.15601748, 0.12244636, 0.68060961, 3.84554221, 0.87326287]),
  0.4216537487305415),
 (array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  array([4.24272338, 0.04182306, 0.70054421, 3.22343662, 1.11963879]),
  0.42499225830722237)]


Result of violate search ...
Time consumption:0.782447099685669


[[array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  0.0],
 [array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  array([4.15601748, 0.12244636, 0.68060961, 3.84554221, 0.87326287]),
  0.4216537487305415],
 [array([4.03575737, 0.21556775, 0.58024099, 3.52590948, 1.07922126]),
  array([4.24272338, 0.04182306, 0.70054421, 3.22343662, 1.11963879]),
  0.42499225830722237]]

In [243]:
X_is = gen_data(0, 5, 25, 5, seed=560)

fail_sum, succ_sum = 0, 0

for idx, X_i in enumerate(X_is):

    rst1 = exhuast_search(X_i, X_train)[:3]
    kd_tree.search_knearest(X_i)
    rst2 = [[X_i, x[0].split, x[1]] for x in kd_tree.kbest]
    
    if sum([str(xx) == str(yy) for xx, yy in zip(rst1, rst2)]) != 3:
        fail_sum += 1
    else:
        succ_sum += 1
        
print("Test Result: All samples num is {}, Correct/Failure:{}/{}".format(fail_sum + succ_sum, succ_sum, fail_sum))
        
    

Test Result: All samples num is 25, Correct/Failure:25/0


In [248]:
%%timeit

X_is = gen_data(0, 5, 1, 5)
  
kd_tree.search_knearest(X_i)
rst2 = kd_tree.kbest

2.65 ms ± 57.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [249]:
%%timeit

X_is = gen_data(0, 5, 1, 5)
  
rst1 = exhuast_search(X_i, X_train)[:3]

644 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
